In [33]:
import psycopg2
import pandas as pd
from psycopg2.extras import execute_values

In [34]:
conn = psycopg2.connect(
    dbname="Habit_Hero_DB",
    user="postgres",       
    password="1151",  
    host="localhost",
    port="5432"
)

cur = conn.cursor()

In [35]:
cur.execute("""
CREATE TABLE IF NOT EXISTS users (
    user_id TEXT PRIMARY KEY,
    signup_date DATE,
    channel TEXT,
    country TEXT,
    device TEXT
);
""")

In [36]:
cur.execute("""
CREATE TABLE IF NOT EXISTS transactions (
    transaction_id TEXT,
    user_id TEXT,
    transaction_date DATE, 
    amount NUMERIC(10,2), 
    type TEXT,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);
""")

In [37]:
cur.execute("""
CREATE TABLE IF NOT EXISTS  engagement (
    user_id TEXT,
    activity_date DATE,
    habits_logged INTEGER,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);
""")

In [38]:
cur.execute("""
CREATE TABLE IF NOT EXISTS costs (
    date DATE,
    channel TEXT,
    cost NUMERIC(10,2)
);
""")

In [39]:
costs_df = pd.read_csv("habit_hero_data/costs.csv")
engagement_df = pd.read_csv("habit_hero_data/engagement.csv")
transactions_df = pd.read_csv("habit_hero_data/transactions.csv")
users_df = pd.read_csv("habit_hero_data/users.csv")

In [40]:
execute_values(
    cur,
    "INSERT INTO users (user_id, signup_date, channel, country, device) VALUES %s",
    [tuple(x) for x in users_df.to_numpy()]
)

In [41]:
execute_values(
    cur,
    "INSERT INTO transactions (transaction_id, user_id, transaction_date, amount, type) VALUES %s",
    [tuple(x) for x in transactions_df.to_numpy()]
)

In [42]:
execute_values(
    cur,
    "INSERT INTO engagement (user_id, activity_date, habits_logged) VALUES %s",
    [tuple(x) for x in engagement_df.to_numpy()]
)

In [43]:
execute_values(
    cur,
    "INSERT INTO costs (date, channel, cost) VALUES %s",
    [tuple(x) for x in costs_df.to_numpy()]
)

In [44]:
conn.commit()

In [45]:
cur.close()
conn.close()